In [1]:
import numpy as np
import pandas as pd
from sklearn import linear_model
import matplotlib.pyplot as plt
from sqlalchemy import create_engine
import statsmodels.api as sm
import seaborn as sns
from scipy.stats import zscore
from scipy.stats.mstats import winsorize
import statsmodels.api as sm
from sklearn.preprocessing import LabelEncoder
from scipy.stats import bartlett
from scipy.stats import levene
from statsmodels.tsa.stattools import acf
from scipy.stats import jarque_bera
from scipy.stats import normaltest

import warnings
warnings.filterwarnings('ignore')

postgres_user = 'dsbc_student'
postgres_pw = '7*.8G9QH21'
postgres_host = '142.93.121.174'
postgres_port = '5432'
postgres_db = 'houseprices'

In [2]:
engine = create_engine('postgresql://{}:{}@{}:{}/{}'.format(
    postgres_user, postgres_pw, postgres_host, postgres_port, postgres_db))
df = pd.read_sql_query('select * from houseprices', con=engine)

#engine.dispose()

df.head()

,id,mssubclass,mszoning,lotfrontage,lotarea,street,alley,lotshape,landcontour,utilities,...,poolarea,poolqc,fence,miscfeature,miscval,mosold,yrsold,saletype,salecondition,saleprice
0,1,60,RL,65.0,8450,Pave,None,Reg,Lvl,AllPub,...,0,None,None,None,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,None,Reg,Lvl,AllPub,...,0,None,None,None,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,None,IR1,Lvl,AllPub,...,0,None,None,None,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,None,IR1,Lvl,AllPub,...,0,None,None,None,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,None,IR1,Lvl,AllPub,...,0,None,None,None,0,12,2008,WD,Normal,250000


### Run your house prices model again.

In [3]:
features = ['overallqual', 'yearbuilt', 'yearremodadd', 'totalbsmtsf', 'firstflrsf', 'grlivarea', 'fullbath', 'totrmsabvgrd', 'garagecars', 'garagearea']

In [4]:
Y = df.saleprice
X = df[features]

In [5]:
X = sm.add_constant(X)

results = sm.OLS(Y, X).fit()

results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              saleprice   R-squared:                       0.774
Model:                            OLS   Adj. R-squared:                  0.772
Method:                 Least Squares   F-statistic:                     495.4
Date:                Thu, 31 Oct 2019   Prob (F-statistic):               0.00
Time:                        14:25:45   Log-Likelihood:                -17459.
No. Observations:                1460   AIC:                         3.494e+04
Df Residuals:                    1449   BIC:                         3.500e+04
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
================================================================================
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const        -1.186e+06   1.29e+05     -9.187      0.000   -1.44e+06   -9.33e+05
overallqual    1.96e+04   1190.159     16.472      0.000    1.73e+04    2.19e+04
yearbuilt      268.2407     50.346      5.328      0.000     169.481     367.000
yearremodadd   296.4812     63.635      4.659      0.000     171.655     421.307
totalbsmtsf     19.8651      4.295      4.625      0.000      11.439      28.291
firstflrsf      14.1737      4.930      2.875      0.004       4.504      23.844
grlivarea       51.2971      4.233     12.119      0.000      42.994      59.600
fullbath     -6790.8715   2682.369     -2.532      0.011   -1.21e+04   -1529.130
totrmsabvgrd    33.1051   1119.061      0.030      0.976   -2162.048    2228.258
garagecars    1.042e+04   3044.272      3.422      0.001    4446.249    1.64e+04
garagearea      14.9475     10.311      1.450      0.147      -5.279      35.174
==============================================================================
Omnibus:                      477.814   Durbin-Watson:                   1.983
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            58906.279
Skew:                          -0.412   Prob(JB):                         0.00
Kurtosis:                      34.107   Cond. No.                     4.69e+05
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 4.69e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

### Assess the goodness of fit of your model using F-test, R-squared, adjusted R-squared, AIC and BIC.

F-statistic is 495.4, R-squared is 0.774, adjusted R-squared is 0.772, AIC is 3.494e+04, BIC is 3.500e+04.

### Do you think your model is satisfactory? If so, why?

idk

### In order to improve the goodness of fit of your model, try different model specifications by adding or removing some variables. For each model you try, get the goodness of fit metrics and compare your models with each other. 

In [6]:
# removing half of features (least correlated)

features.remove('totrmsabvgrd')
features.remove('yearremodadd')
features.remove('yearbuilt')
features.remove('fullbath')
features.remove('totalbsmtsf')
features.remove('firstflrsf')

In [7]:
Y = df.saleprice
X = df[features]

In [8]:
X = sm.add_constant(X)

results = sm.OLS(Y, X).fit()

results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              saleprice   R-squared:                       0.742
Model:                            OLS   Adj. R-squared:                  0.741
Method:                 Least Squares   F-statistic:                     1045.
Date:                Thu, 31 Oct 2019   Prob (F-statistic):               0.00
Time:                        14:25:45   Log-Likelihood:                -17556.
No. Observations:                1460   AIC:                         3.512e+04
Df Residuals:                    1455   BIC:                         3.515e+04
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
===============================================================================
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
const       -9.844e+04   4820.467    -20.420      0.000   -1.08e+05    -8.9e+04
overallqual  2.699e+04   1067.393     25.285      0.000    2.49e+04    2.91e+04
grlivarea      49.5728      2.555     19.402      0.000      44.561      54.585
garagecars   1.132e+04   3126.297      3.620      0.000    5184.992    1.75e+04
garagearea     41.4784     10.627      3.903      0.000      20.632      62.324
==============================================================================
Omnibus:                      411.179   Durbin-Watson:                   1.988
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            12305.361
Skew:                           0.670   Prob(JB):                         0.00
Kurtosis:                      17.159   Cond. No.                     7.73e+03
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 7.73e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

### Combining correlated features (> .80)

In [37]:
# combining correlated features (> .80)

# yearbuilt x garageyrbuilt
# grlivarea x totrmsabvgrd
# firstflrsf x totalbsmtsf
# garagearea x garagecars

# reset feature list
features = ['overallqual', 'yearbuilt', 'yearremodadd', 'totalbsmtsf', 'firstflrsf', 'grlivarea', 'fullbath', 'totrmsabvgrd', 'garagecars', 'garagearea']

#df['yearbuilt_garageyrblt'] = df.yearbuilt * df.garageyrblt
#features.append('yearbuilt_garageyrblt')
#features.remove('yearbuilt')
#features.remove('garageyrblt')

df['grlivarea_totrmsabvgrd'] = df.grlivarea * df.totrmsabvgrd
features.append('grlivarea_totrmsabvgrd')
features.remove('grlivarea')
features.remove('totrmsabvgrd')

df['firstflrsf_totalbsmtsf'] = df.firstflrsf * df.totalbsmtsf
features.append('firstflrsf_totalbsmtsf')
features.remove('firstflrsf')
features.remove('totalbsmtsf')

df['garagearea_garagecars'] = df.garagearea * df.garagecars
features.append('garagearea_garagecars')
features.remove('garagearea')
features.remove('garagecars')

In [38]:
Y = df.saleprice
X = df[features]

In [43]:
X = sm.add_constant(X)

results = sm.OLS(Y, X).fit()

results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              saleprice   R-squared:                       0.793
Model:                            OLS   Adj. R-squared:                  0.791
Method:                 Least Squares   F-statistic:                     616.3
Date:                Thu, 31 Oct 2019   Prob (F-statistic):               0.00
Time:                        14:38:00   Log-Likelihood:                -17395.
No. Observations:                1460   AIC:                         3.481e+04
Df Residuals:                    1450   BIC:                         3.486e+04
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
==========================================================================================
                             coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------
const                  -1.054e+06   1.23e+05     -8.564      0.000    -1.3e+06   -8.12e+05
overallqual             1.781e+04   1134.911     15.694      0.000    1.56e+04       2e+04
yearbuilt                241.2862     47.068      5.126      0.000     148.957     333.616
yearremodadd             256.8123     61.108      4.203      0.000     136.943     376.682
fullbath               -8158.9015   2509.833     -3.251      0.001   -1.31e+04   -3235.609
grlivarea_totrmsabvgrd     0.1926      0.333      0.578      0.563      -0.461       0.846
firstflrsf_totalbsmtsf    -0.0142      0.002     -8.526      0.000      -0.017      -0.011
garagearea_garagecars     19.3160      1.824     10.592      0.000      15.739      22.893
inside_sf                 57.5623      4.252     13.539      0.000      49.222      65.902
outside_sf                29.7932      7.294      4.085      0.000      15.486      44.101
==============================================================================
Omnibus:                      499.247   Durbin-Watson:                   1.956
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            16902.024
Skew:                           0.927   Prob(JB):                         0.00
Kurtosis:                      19.565   Cond. No.                     2.39e+08
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.39e+08. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [44]:
# reset feature list
#features = ['overallqual', 'yearbuilt', 'yearremodadd', 'grlivarea', 'fullbath', 'totrmsabvgrd', 'garagecars', 'garagearea']

df['inside_sf'] = df.totalbsmtsf + df.firstflrsf + df.secondflrsf
features.append('inside_sf')

df['outside_sf'] = df.wooddecksf + df.openporchsf
features.append('outside_sf')

#features.remove('firstflrsf_totalbsmtsf')

features

['overallqual',
 'yearbuilt',
 'yearremodadd',
 'fullbath',
 'grlivarea_totrmsabvgrd',
 'firstflrsf_totalbsmtsf',
 'garagearea_garagecars',
 'inside_sf',
 'outside_sf',
 'inside_sf',
 'outside_sf']

In [45]:
Y = df.saleprice
X = df[features]

In [50]:
X = sm.add_constant(X)

results = sm.OLS(Y, X).fit()

results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              saleprice   R-squared:                       0.793
Model:                            OLS   Adj. R-squared:                  0.791
Method:                 Least Squares   F-statistic:                     616.3
Date:                Thu, 31 Oct 2019   Prob (F-statistic):               0.00
Time:                        14:38:24   Log-Likelihood:                -17395.
No. Observations:                1460   AIC:                         3.481e+04
Df Residuals:                    1450   BIC:                         3.486e+04
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
==========================================================================================
                             coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------
const                  -1.054e+06   1.23e+05     -8.564      0.000    -1.3e+06   -8.12e+05
overallqual             1.781e+04   1134.911     15.694      0.000    1.56e+04       2e+04
yearbuilt                241.2862     47.068      5.126      0.000     148.957     333.616
yearremodadd             256.8123     61.108      4.203      0.000     136.943     376.682
fullbath               -8158.9015   2509.833     -3.251      0.001   -1.31e+04   -3235.609
grlivarea_totrmsabvgrd     0.1926      0.333      0.578      0.563      -0.461       0.846
firstflrsf_totalbsmtsf    -0.0142      0.002     -8.526      0.000      -0.017      -0.011
garagearea_garagecars     19.3160      1.824     10.592      0.000      15.739      22.893
inside_sf                 28.4438      3.956      7.190      0.000      20.684      36.204
outside_sf                 0.6747      5.230      0.129      0.897      -9.585      10.935
total_sf                  29.1185      2.658     10.953      0.000      23.904      34.333
==============================================================================
Omnibus:                      499.247   Durbin-Watson:                   1.956
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            16902.024
Skew:                           0.927   Prob(JB):                         0.00
Kurtosis:                      19.565   Cond. No.                     1.08e+18
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is 4.25e-21. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""

In [51]:
df['total_sf'] = df.totalbsmtsf + df.firstflrsf + df.secondflrsf + df.wooddecksf + df.openporchsf
features.append('total_sf')
features.remove('inside_sf')
features.remove('outside_sf')

features

['overallqual',
 'yearbuilt',
 'yearremodadd',
 'fullbath',
 'grlivarea_totrmsabvgrd',
 'firstflrsf_totalbsmtsf',
 'garagearea_garagecars',
 'total_sf',
 'total_sf']

In [52]:
Y = df.saleprice
X = df[features]

In [53]:
X = sm.add_constant(X)

results = sm.OLS(Y, X).fit()

results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              saleprice   R-squared:                       0.791
Model:                            OLS   Adj. R-squared:                  0.790
Method:                 Least Squares   F-statistic:                     688.0
Date:                Thu, 31 Oct 2019   Prob (F-statistic):               0.00
Time:                        14:38:30   Log-Likelihood:                -17400.
No. Observations:                1460   AIC:                         3.482e+04
Df Residuals:                    1451   BIC:                         3.487e+04
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
==========================================================================================
                             coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------
const                  -9.998e+05   1.22e+05     -8.182      0.000   -1.24e+06    -7.6e+05
overallqual             1.833e+04   1126.179     16.274      0.000    1.61e+04    2.05e+04
yearbuilt                233.6717     47.147      4.956      0.000     141.188     326.155
yearremodadd             240.0008     61.054      3.931      0.000     120.236     359.765
fullbath               -7489.1119   2508.211     -2.986      0.003   -1.24e+04   -2569.006
grlivarea_totrmsabvgrd     0.5573      0.313      1.781      0.075      -0.057       1.171
firstflrsf_totalbsmtsf    -0.0119      0.001     -7.937      0.000      -0.015      -0.009
garagearea_garagecars     19.5148      1.828     10.675      0.000      15.929      23.101
total_sf                  24.9275      1.737     14.352      0.000      21.520      28.335
total_sf                  24.9275      1.737     14.352      0.000      21.520      28.335
==============================================================================
Omnibus:                      486.374   Durbin-Watson:                   1.955
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            19572.144
Skew:                           0.831   Prob(JB):                         0.00
Kurtosis:                      20.860   Cond. No.                     6.99e+18
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is 1.02e-22. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""

### For each model you try, get the goodness of fit metrics and compare your models with each other. Which model is the best and why?

So far, the last model is the best, since it has the highest R-squared scores.